In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Importing libs
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from textwrap import wrap
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import plotly as py
import plotly.graph_objs as go
import os
py.offline.init_notebook_mode(connected = True)
#print(os.listdir("../input"))
import datetime as dt

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.gridspec as grid_spec
import seaborn as sns
from imblearn.over_sampling import SMOTE
import scikitplot as skplt


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split,cross_val_score


from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, precision_score, f1_score
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler
# data splitting
from sklearn.model_selection import train_test_split
# data modeling
from sklearn.metrics import confusion_matrix,accuracy_score,roc_curve,classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
#ensembling
from mlxtend.classifier import StackingCVClassifier

# Tabular Playgroung Series [Apr 2021]

Welcome...

This dataset is based on the classic Titanic dataset, but is synthetic and generated using a CTGAN.

**Can we predict who survives, and who perishes?**

# Exploratory Data Analysis

Firstly, I will explore the data. I am a Kaggler who loves data visualisation so this will be a main focus of this project.

# Models

Next, I will use several models to see which performs the best. I will also then stack these models together.



# Work in progress

I will continue to add visualisations to this project over the coming days & weeks...

In [ ]:
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/train.csv')
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/test.csv')

print(train_df.shape)
print(test_df.shape)

In [ ]:
train_df.head(3)

In [ ]:
# Custom parameters for my plots

from matplotlib import rc
font = {'size'   : 10}
matplotlib.rc('font', **font)
# change font
matplotlib.rcParams['font.sans-serif'] = "Times New Roman"
matplotlib.rcParams['font.family'] = "serif"


# colors

background_color = '#fff1e5'
face_color = '#fffcfa'

col = '#33302e'
sub_col = '#a49a93'

# Custom colour map 
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ['#1e558b','#FF8AAF','#E6668E'])

surv_c = '#1e558b'
no_surv_c = '#E6668E'


title = 15
sub = 10
font = 'serif'
b = 'bold'

col = '#33302e'
sub_col = '#a49a93'

# Custom colour map 
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ['#1e558b','#FF8AAF','#E6668E'])


# Let's get a feel for the data

In [ ]:
missing = pd.DataFrame(columns=['% Missing values'],data=train_df.isnull().sum()/len(train_df))
missing.drop(['Survived'],inplace=True)

missing_tst = pd.DataFrame(columns=['% Missing values'],data=test_df.isnull().sum()/len(test_df))


fig = plt.figure(figsize=(5, 5), dpi=150,facecolor=background_color)
gs = fig.add_gridspec(1, 2)
gs.update(wspace=0.02, hspace=0)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[0, 1])

ax0.set_facecolor(face_color)
ax1.set_facecolor(face_color)


for s in ["right", "top","bottom","left"]:
    ax0.spines[s].set_visible(False)
    ax1.spines[s].set_visible(False)
    
sns.heatmap(missing,cbar=False,cmap=cmap,annot=True,fmt=".1%", linewidths=2.5,vmax=0.1, ax=ax0)

sns.heatmap(missing_tst,cbar=False,cmap=cmap,annot=True,fmt=".1%", linewidths=2,vmax=0.1, ax=ax1)

ax0.tick_params(axis=u'both', which=u'both',length=0)
ax1.tick_params(axis=u'both', which=u'both',length=0)

# remove y labels for second plot
ax1.set_yticklabels([])
ax0.set_xticklabels([])
ax1.set_xticklabels([])


# Ax lens

Xstart, Xend = ax0.get_xlim()
Ystart, Yend = ax0.get_ylim()

Xstart1, Xend1 = ax1.get_xlim()
Ystart1, Yend1 = ax1.get_ylim()

# title

ax0.text(Xstart,Yend-1.7,'How much data is missing?',color=col,fontsize=title,fontweight=b)
ax0.text(Xstart,Yend-0.6,'There are many missing values,\nparticularly in the Cabin field.',color=sub_col,fontsize=sub,fontweight=b)

ax0.text(Xstart,Ystart+0.5,'Train',color=sub_col,fontsize=sub,fontweight=b)
ax1.text(Xend1-0.185,Ystart1+0.5,'Test',color=sub_col,fontsize=sub,fontweight=b)


plt.show()

# We'll deal with missing data shortly

Before I do, I want to start undertsanding what it is I'd be imputing. Is it necessary? Or can I drop the data?

In [ ]:
feature_cols = train_df.drop(['Survived','PassengerId'], axis=1).columns
target_column = 'Survived'
numerical_columns = train_df[feature_cols].select_dtypes(include=['int64','float64']).columns
categorical_columns = train_df[feature_cols].select_dtypes(exclude=['int64','float64']).columns


fig = plt.figure(figsize=(8, 5), dpi=150,facecolor=background_color)
gs = fig.add_gridspec(1, 1)
ax0 = fig.add_subplot(gs[0, 0])

ax0.set_facecolor(face_color)
ax0.text(-1.1, 0.26, 'Correlation of Continuous Features with Target', fontsize=title, fontweight=b, color=col)
ax0.text(-1.1, 0.24, 'It is clear here that class has a big imapct on survival and so does fare paid - though these are likely related.' ,fontsize=sub, color=sub_col)

chart_df = pd.DataFrame(train_df[numerical_columns].corrwith(train_df['Survived']))
chart_df.columns = ['corr']
chart_df['positive'] = chart_df['corr'] > 0

sns.barplot(x=chart_df.index, y=chart_df['corr'], ax=ax0, palette=chart_df.positive.map({True: surv_c, False: no_surv_c}), zorder=3,dodge=False)
ax0.grid(which='major', axis='y', zorder=0, color='gray', linestyle=':', dashes=(1,5))
ax0.set_ylabel('')


for s in ["top","right", 'left']:
    ax0.spines[s].set_visible(False)
    
ax0.annotate('-0.29',xy=(0,-0.29-(-0.289723*0.1)),ha='center',color='white')
ax0.annotate('0.19',xy=(4,0.19-(0.187376*0.12)),ha='center',color='white')

plt.show()

# How related are these features to each other?

In [ ]:
fig = plt.figure(figsize=(12, 8), facecolor=background_color)
gs = fig.add_gridspec(1, 2)
gs.update(wspace=0, hspace=0.27)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[0, 1])
colors = ["#fbfbfb", "lightgray","#0e4f66"]
colormap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)

ax0.set_facecolor(background_color)
ax0.text(0, 0, 'Train', fontsize=20, fontweight='bold', fontfamily='serif',color='lightgray')

ax1.set_facecolor(background_color)
ax1.text(4.5, 5.5, 'Test', fontsize=20, fontweight='bold', fontfamily='serif',color='lightgray')


fig.text(0.5,0.5,'Correlation of Features\nFor Train & Test\nDatasets', fontsize=title+2, fontweight='bold', color=col,va='center',ha='center')

corr = train_df[numerical_columns].corr().abs()
mask = np.triu(np.ones_like(corr, dtype=np.bool))
sns.heatmap(corr, ax=ax0, vmin=0, vmax=1, annot=True, square=True, mask=mask,
            cbar_kws={"orientation": "horizontal"}, cbar=False, cmap=cmap, fmt='.1g',linewidth=3,linecolor=background_color)


corr = test_df[numerical_columns].corr().abs()
mask = np.tril(corr)
sns.heatmap(corr, ax=ax1, vmin=0, vmax=1, annot=True, square=True, mask=mask,
            cbar_kws={"orientation": "horizontal"}, cbar=False, cmap=cmap, fmt='.1g',linewidth=3,linecolor=background_color)
ax1.xaxis.tick_top()
ax1.yaxis.tick_right()

plt.show()

As we suspected, Fare and Class are highly related to eachother

In [ ]:
fig = plt.figure(figsize=(8, 3), dpi=150,facecolor=background_color)
gs = fig.add_gridspec(2, 1)
gs.update(wspace=0.02, hspace=0.5)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
ax0.set_facecolor(face_color)

for s in ["right", "top","bottom","left"]:
    ax0.spines[s].set_visible(False)
    ax1.spines[s].set_visible(False)


surv = train_df[train_df['Survived'] == 1]
no_surv = train_df[train_df['Survived'] == 0]

sns.kdeplot(surv['Fare'],color=surv_c,shade=True,ax=ax0)
sns.kdeplot(no_surv['Fare'],color=no_surv_c,shade=True,ax=ax0)

sns.kdeplot(surv['Age'],color=surv_c,shade=True,ax=ax1)
sns.kdeplot(no_surv['Age'],color=no_surv_c,shade=True,ax=ax1)

ax0.tick_params(axis=u'both', which=u'both',length=0)
ax1.tick_params(axis=u'both', which=u'both',length=0)


 #remove y labels for second plot
ax0.set_yticklabels([])
ax1.set_yticklabels([])

ax0.tick_params(axis='x', which='major', labelsize=8)
ax0.tick_params(axis='x', colors=sub_col)

ax1.tick_params(axis='x', which='major', labelsize=8)
ax1.tick_params(axis='x', colors=sub_col)
                          
# Ax lens

Xstart, Xend = ax0.get_xlim()
Ystart, Yend = ax0.get_ylim()

ax0.set_xlabel("Fare",loc='left',fontsize=sub,color=sub_col)
ax0.set_ylabel(" ",loc='top',fontsize=sub,color=sub_col)


ax1.set_xlabel("Age",loc='left',fontsize=sub,color=sub_col)
ax1.set_ylabel(" ",loc='top',fontsize=sub,color=sub_col)

ax0.text(Xstart,Yend+(Yend*0.6),'Are there differences in survival rates by Age & Fare?',fontsize=title,fontweight=b,color=col)
ax0.text(Xstart,Yend+(Yend*0.2),
       
'''
We see a noticeable differences between those who survived and those who did not.
''',color=sub_col,fontsize=sub)

ax0.text(Xstart,Yend+0.005,'Survivors',fontsize=sub-2,fontweight=b,color=surv_c)
ax0.text(Xstart+85,Yend+0.005,'|',fontsize=sub-2,fontweight=b,color=col)
ax0.text(Xstart+90,Yend+0.005,'Non-Survivors',fontsize=sub-2,fontweight=b,color=no_surv_c)


# Dealing with missing values

I have now seen that there is seemingly relationships between Age, Fare paid, and survivorship rates.

The plots above also reveal that the Fare in particular is very positively skewed. As a result, I will impute the median values as opposed to the mean.



In [ ]:
train_df['Age'].fillna(train_df['Age'].median(), inplace=True)
test_df['Age'].fillna(test_df['Age'].median(), inplace=True)
train_df['Fare'].fillna(train_df['Fare'].median(), inplace=True)
test_df['Fare'].fillna(test_df['Fare'].median(), inplace=True)

# We aren't done with missing values yet, but first I want to examine more what factors seem to be related to our primary variable: Survived, or did not survive

Is our dataset balanced?

If not, we will utulise methods such as SMOTE

In [ ]:
x=train_df.groupby(['Survived'])['Survived'].count()
y=len(train_df)
r=((x/y)).round(2)

ratio = pd.DataFrame(r).T

fig = plt.figure(figsize=(8, 10), dpi=150,facecolor=background_color)
gs = fig.add_gridspec(5, 1)
gs.update(wspace=0.02, hspace=0.8)
ax = fig.add_subplot(gs[0, 0])



ax.set_facecolor(background_color)

ax.barh(ratio.index, ratio[1.0], color=surv_c, alpha=0.9, label='Survived')
ax.barh(ratio.index, ratio[0.0], left=ratio[1.0], color=no_surv_c, alpha=0.9, label='Perished')

ax.set_xlim(0, 1)
ax.set_xticks([])
ax.set_yticks([])
#ax.set_yticklabels(mf_ratio.index, fontfamily='serif', fontsize=11)


# movie percentage
for i in ratio.index:
    ax.annotate(f"{int(ratio[1.0][i]*100)}%", 
                   xy=(ratio[1.0][i]/2, i),
                   va = 'center', ha='center',fontsize=32, fontweight='light', fontfamily='serif',
                   color='white')

    ax.annotate("Survived", 
                   xy=(ratio[1.0][i]/2, -0.25),
                   va = 'center', ha='center',fontsize=12, fontweight='light', fontfamily='serif',
                   color='white')
    
    
for i in ratio.index:
    ax.annotate(f"{int(ratio[0.0][i]*100)}%", 
                   xy=(ratio[1.0][i]+ratio[0.0][i]/2, i),
                   va = 'center', ha='center',fontsize=32, fontweight='light', fontfamily='serif',
                   color='white')
    ax.annotate("Perished", 
                   xy=(ratio[1.0][i]+ratio[0.0][i]/2, -0.25),
                   va = 'center', ha='center',fontsize=12, fontweight='light', fontfamily='serif',
                   color='white')



# Title & Subtitle
ax.text(0,0.75,'How many survived & did not survive in our dataset?',fontsize=title, fontweight=b,color=col)
ax.text(0,0.6,'We see a fairly balanced dataset, with slightly less survivors in total.',fontsize=sub,color=sub_col)  

for s in ['top', 'left', 'right', 'bottom']:
    ax.spines[s].set_visible(False)
    
#ax.legend(loc='lower center', ncol=3, bbox_to_anchor=(0.5, -0.06))

# Removing legend due to labelled plot
ax.legend().set_visible(False)
plt.show()

Our dataset is balanced, meaning we have roughly equal number of each target variable.

# Overview of categorical features

In [ ]:
fig = plt.figure(figsize=(4, 10), dpi=150,facecolor=background_color)
gs = fig.add_gridspec(5, 1)
gs.update(wspace=0.02, hspace=0.8)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
ax2 = fig.add_subplot(gs[2, 0])


ax0.set_facecolor(face_color)
ax1.set_facecolor(face_color)
ax2.set_facecolor(face_color)

for s in ["right", "top","bottom","left"]:
    ax0.spines[s].set_visible(False)
    ax1.spines[s].set_visible(False)
    ax2.spines[s].set_visible(False)

#plots
sns.countplot(x=train_df['Sex'],hue=train_df['Survived'],edgecolor=face_color,palette=[no_surv_c,surv_c],ax=ax0)
ax0.set_xlabel("Sex",loc='left',fontsize=sub,color=sub_col)
ax0.set_ylabel(" ",loc='top',fontsize=sub,color=sub_col)
ax0.get_legend().remove()

sns.countplot(x=train_df['Pclass'],hue=train_df['Survived'],edgecolor=face_color,palette=[no_surv_c,surv_c],ax=ax1)
ax1.set_xlabel("Class",loc='left',fontsize=sub,color=sub_col)
ax1.set_ylabel(" ",loc='top',fontsize=sub,color=sub_col)
ax1.get_legend().remove()

sns.countplot(x=train_df['Embarked'],hue=train_df['Survived'],edgecolor=face_color,palette=[no_surv_c,surv_c],ax=ax2)
ax2.set_xlabel("Embarkation Port",loc='left',fontsize=sub,color=sub_col)
ax2.set_ylabel(" ",loc='top',fontsize=sub,color=sub_col)
ax2.get_legend().remove()

Xstart, Xend = ax0.get_xlim()
Ystart, Yend = ax0.get_ylim()
ax0.set_yticks(np.arange(0, Yend, 10000))

ax0.text(Xstart,100000,'Where do the total deaths come from?',fontsize=title,fontweight=b,color=col)
ax0.text(Xstart,81500,'We observe areas that were incredibly unlikley to survive\nfor example, males were far more likely to perish',fontsize=sub,color=sub_col)

ax0.text(Xend+0.5,15000,
'''
Males were far more likely to 
perish than females.
This will no doubt be important
for our model.
''',fontsize=sub-2,color=sub_col)


ax0.set_yticklabels([])
ax0.tick_params(axis='x', which='major', labelsize=8)
ax0.tick_params(axis='both', colors=sub_col)
ax0.tick_params(axis=u'both', which=u'both',length=0)

Xstart, Xend = ax1.get_xlim()
Ystart, Yend = ax1.get_ylim()


ax1.text(Xend+0.75,10000,
'''
Unsuprisingly those in the
lower classes persihed at
much higher rates than those
in higher classes.
''',fontsize=sub-2,color=sub_col)


ax1.set_yticks(np.arange(0, Yend, 10000))

ax1.set_yticklabels([])
ax1.tick_params(axis='x', which='major', labelsize=8)
ax1.tick_params(axis='both', colors=sub_col)
ax1.tick_params(axis=u'both', which=u'both',length=0)

Xstart, Xend = ax2.get_xlim()
Ystart, Yend = ax2.get_ylim()
ax2.set_yticks(np.arange(0, Yend, 10000))

ax2.text(Xend+0.75,15000,
'''
Many of those who perished
boarded in Southampton, but this
was the main embarkation port
so this is not surprising.
''',fontsize=sub-2,color=sub_col)

ax2.set_yticklabels([])
ax2.tick_params(axis='x', which='major', labelsize=8)
ax2.tick_params(axis='both', colors=sub_col)
ax2.tick_params(axis=u'both', which=u'both',length=0)


ax0.text(Xstart,60000,'Survivors',fontsize=sub-2,fontweight=b,color=surv_c)
ax0.text(Xstart+0.38,60000,'|',fontsize=sub-2,fontweight=b,color=col)
ax0.text(Xstart+0.405,60000,'Non-Survivors',fontsize=sub-2,fontweight=b,color=no_surv_c)


import matplotlib.lines as lines
l1 = lines.Line2D([1, 1], [0.4, 0.88], transform=fig.transFigure, figure=fig,color='black',lw=0.2)
fig.lines.extend([l1])

ax0.text(2,60000,'Insights',fontsize=sub-2,fontweight=b,color=col)


plt.show()

In [ ]:
# Function from Subin An 
# https://www.kaggle.com/subinium/tps-apr-highlighting-the-data

def age_band(num):
    for i in range(1, 100):
        if num < 10*i :  return f'{(i-1) * 10} ~ {i*10}'

train_df['Age band'] = train_df['Age'].apply(age_band)
titanic_age = train_df[['Age band', 'Survived','Sex']].groupby('Age band')['Survived'].value_counts().sort_index().unstack().fillna(0)
titanic_age['Survival rate'] = titanic_age[1] / (titanic_age[0] + titanic_age[1]) * 100
age_band = train_df['Age band'].value_counts().sort_index()



We know that sex is a big factor in determining whether or not someone lived or dies, but does this hold true across all age ranges?

In [ ]:
age_sex_surv = train_df.groupby(['Sex','Age band'])['Survived'].mean().unstack().T
fem_mean = age_sex_surv['female'].mean()
male_mean = age_sex_surv['male'].mean()

fig = plt.figure(figsize=(10, 4), dpi=150,facecolor=background_color)
gs = fig.add_gridspec(1, 1)
gs.update(wspace=0.2, hspace=0.8)
ax0 = fig.add_subplot(gs[0, 0])

ax0.set_facecolor(face_color)

for s in ["right", "top","bottom","left"]:
    ax0.spines[s].set_visible(False)

my_range=range(1,len(age_sex_surv.index)+1)
 
ax0.hlines(y=my_range, xmin=age_sex_surv['male'], xmax=age_sex_surv['female'], color='gray', alpha=0.4)
sns.scatterplot(age_sex_surv['male'], my_range, color=surv_c, alpha=1,s=100, label='male',ax=ax0)

sns.scatterplot(age_sex_surv['female'], my_range, color=no_surv_c, alpha=1,s=100, label='female',ax=ax0)

ax0.get_legend().remove()

Xstart, Xend = ax0.get_xlim()
Ystart, Yend = ax0.get_ylim()
ax0.set_xticks(np.arange(0, 1, 0.1))
ax0.set_yticklabels([' ','0 ~ 10', '10 ~ 20', '20 ~ 30', '30 ~ 40', '40 ~ 50', '50 ~ 60','60 ~ 70', '70 ~ 80', '80 ~ 90'])


ax0.tick_params(axis='x', which='major', labelsize=8)
ax0.tick_params(axis='both', colors=sub_col)
ax0.tick_params(axis=u'both', which=u'both',length=0)

ax0.set_xlabel("Survival Rate",loc='left',fontsize=sub,color=sub_col)


ax0.text(0,11.2,'Survival rates by age & sex',fontsize=title,fontweight=b,color=col)
ax0.text(0,10.6,'Female out survive males across all age cateogories, and the mean survial rate is much higher',fontsize=sub,color=sub_col)

ax0.text(0,10,'Male',fontsize=sub-2,fontweight=b,color=surv_c)
ax0.text(0+0.037,10,'|',fontsize=sub-2,fontweight=b,color=col)
ax0.text(0+0.0436,10,'Female',fontsize=sub-2,fontweight=b,color=no_surv_c)

#ax0.vlines(x=fem_mean)
ax0.axvline(male_mean ,color=surv_c, linewidth=0.4, linestyle='dashdot')
ax0.axvline(fem_mean ,color=no_surv_c, linewidth=0.4, linestyle='dashdot')


# Show the graph
plt.show()

Yes, it hold true in every age range, and is very significant.

However, we also see that the elderly males stood the best chance of surviving, and interestingly, that young females struggled comparatively.

what about other features?

In [ ]:
# idea from Subin An - though it has been coded differently
# https://www.kaggle.com/subinium/tps-apr-highlighting-the-data

cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ['#E6668E','#FF8AAF','#1e558b'])


fig = plt.figure(figsize=(6, 12), dpi=150,facecolor=background_color)
gs = fig.add_gridspec(1, 2)
gs.update(wspace=0.2, hspace=0.8)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[0, 1])


ax0.set_facecolor(background_color)
ax1.set_facecolor(background_color)


for s in ["right", "top","bottom","left"]:
    ax0.spines[s].set_visible(False)
    ax1.spines[s].set_visible(False)

sns.heatmap(train_df.groupby(['Sex', 'Embarked'])['Survived'].aggregate('mean').unstack()*100, ax=ax0,
                square=True, annot=True, fmt='.2f', linewidth=2,
                cbar=False, cmap=cmap)

sns.heatmap(train_df.groupby(['Sex', 'Pclass'])['Survived'].aggregate('mean').unstack()*100, ax=ax1,
                square=True, annot=True, fmt='.2f', linewidth=2,
                cbar=False, cmap=cmap)

#C = Cherbourg, Q = Queenstown, S = Southampton
#ax0.set_xticklabels(['Cherbourg','Queenstown','Southampton'])
ax1.set_yticklabels([])

ax0.set_yticklabels(['Female','Male'], rotation=0)
ax0.tick_params(axis='x', which='major', labelsize=8)
ax0.tick_params(axis='both', colors=sub_col)
ax0.tick_params(axis=u'both', which=u'both',length=0)
ax0.set_ylabel('') 
ax1.set_ylabel('') 

ax1.tick_params(axis='x', which='major', labelsize=8)
ax1.tick_params(axis='both', colors=sub_col)
ax1.tick_params(axis=u'both', which=u'both',length=0)


ax0.set_xlabel("Embarkation Port",loc='left',fontsize=sub,color=sub_col)
ax1.set_xlabel("Class",loc='left',fontsize=sub,color=sub_col)

ax0.text(0,-1.1,'Survival rates',fontweight=b,fontsize=title,color=col)
ax0.text(0,-0.3,
'''
Here we clearly see the trends indicated above.
Females, in all categories, have high survial rates;
especially those who were in first and second class''',fontsize=sub,color=sub_col)

plt.show()

In [ ]:
new = age_sex_surv.mean()

new = pd.DataFrame(new)

new['no_surv'] = 1-new[0]

#new

from statsmodels.graphics.mosaicplot import mosaic

# in progress


# Models

Here I will try a variety of models to see which performs best on our data

In [ ]:
# model prep

train_df = pd.get_dummies(train_df, columns=["Sex", "Embarked"])
test_df = pd.get_dummies(test_df, columns=["Sex", "Embarked"])

train_df['Fare'] = train_df['Fare'].map(lambda i: np.log(i) if i > 0 else 0)
test_df['Fare'] = test_df['Fare'].map(lambda i: np.log(i) if i > 0 else 0)

In [ ]:
# spltting the data

X = train_df[["Pclass", "Age", "SibSp", "Parch", "Fare", "Sex_male", "Sex_female", "Embarked_C", "Embarked_Q", "Embarked_S"]]
y = train_df['Survived']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 0)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
lr = LogisticRegression()
model = lr.fit(X_train, y_train)
lr_predict = lr.predict(X_test)
lr_conf_matrix = confusion_matrix(y_test, lr_predict)
lr_acc_score = accuracy_score(y_test, lr_predict)
print("confussion matrix")
print(lr_conf_matrix)
print("\n")
print("Accuracy of Logistic Regression:",lr_acc_score*100,'\n')
print(classification_report(y_test,lr_predict))

In [ ]:
lr = LogisticRegression(C=0.01, penalty='l2')
model = lr.fit(X_train, y_train)
lr_predict = lr.predict(X_test)
lr_conf_matrix = confusion_matrix(y_test, lr_predict)
lr_acc_score = accuracy_score(y_test, lr_predict)
print("confusion matrix")
print(lr_conf_matrix)
print("\n")
print("Accuracy of Logistic Regression:",lr_acc_score*100,'\n')
print(classification_report(y_test,lr_predict))

If we manipulate the threshold our logisitc model uses to determine 0 or 1, can we improve the score?

In [ ]:
#source code: https://www.kaggle.com/prashant111/extensive-analysis-eda-fe-modelling
# modified

from sklearn.preprocessing import binarize

for i in range(1,8):
    
    cm1=0
    y_pred1 = lr.predict_proba(X_test)[:,1]
    y_pred1 = y_pred1.reshape(-1,1)
    y_pred2 = binarize(y_pred1, i/10)
    y_pred2 = np.where(y_pred2 == 1, 1, 0)
    cm1 = confusion_matrix(y_test, y_pred2)
        
    print ('With',i/10,'threshold the Confusion Matrix is ','\n\n',cm1,'\n\n',
            'with',cm1[0,0]+cm1[1,1],'correct predictions, ', '\n\n', 
           
            cm1[0,1],'Type I errors( False Positives), ','\n\n',
           
            cm1[1,0],'Type II errors( False Negatives), ','\n\n',
           
           'Accuracy score: ', (accuracy_score(y_test, y_pred2)), '\n\n',
          
            '====================================================', '\n\n')

On this case, a threshold of 0.5 still gives the best results

In [ ]:
nb = GaussianNB()
nb.fit(X_train,y_train)
nbpred = nb.predict(X_test)
nb_conf_matrix = confusion_matrix(y_test, nbpred)
nb_acc_score = accuracy_score(y_test, nbpred)
print("confusion matrix")
print(nb_conf_matrix)
print("\n")
print("Accuracy of Naive Bayes model:",nb_acc_score*100,'\n')
print(classification_report(y_test,nbpred))

In [ ]:
rf = RandomForestClassifier(n_estimators=20, random_state=12,max_depth=7)
rf.fit(X_train,y_train)
rf_predicted = rf.predict(X_test)
rf_conf_matrix = confusion_matrix(y_test, rf_predicted)
rf_acc_score = accuracy_score(y_test, rf_predicted)
print("confusion matrix")
print(rf_conf_matrix)
print("\n")
print("Accuracy of Random Forest:",rf_acc_score*100,'\n')
print(classification_report(y_test,rf_predicted))

In [ ]:
xgb = XGBClassifier(learning_rate=0.1, n_estimators=64, max_depth=15,gamma=0.6, subsample=0.52,colsample_bytree=0.6,seed=27, 
                    reg_lambda=2, booster='dart', colsample_bylevel=0.6, colsample_bynode=0.5)
xgb.fit(X_train, y_train)
xgb_predicted = xgb.predict(X_test)
xgb_conf_matrix = confusion_matrix(y_test, xgb_predicted)
xgb_acc_score = accuracy_score(y_test, xgb_predicted)
print("confusion matrix")
print(xgb_conf_matrix)
print("\n")
print("Accuracy of Extreme Gradient Boost:",xgb_acc_score*100,'\n')
print(classification_report(y_test,xgb_predicted))

In [ ]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)
knn_predicted = knn.predict(X_test)
knn_conf_matrix = confusion_matrix(y_test, knn_predicted)
knn_acc_score = accuracy_score(y_test, knn_predicted)
print("confusion matrix")
print(knn_conf_matrix)
print("\n")
print("Accuracy of K-NeighborsClassifier:",knn_acc_score*100,'\n')
print(classification_report(y_test,knn_predicted))

That is a lot of information to digest. Let's pull it all together and view it simultaneously

In [ ]:
model_eval = pd.DataFrame({'Model': ['Logistic Regression','Naive Bayes','Random Forest','XGBoost',
                    'K-Nearest Neighbour'], 'Accuracy': [lr_acc_score,
                    nb_acc_score,rf_acc_score,xgb_acc_score,knn_acc_score]})

model_eval = model_eval.set_index('Model').sort_values(by='Accuracy',ascending=False)


cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ['#E6668E','#FF8AAF','#1e558b'])


fig = plt.figure(figsize=(12, 6), dpi=150,facecolor=background_color)
gs = fig.add_gridspec(1, 2)
gs.update(wspace=0.2, hspace=0.8)
ax0 = fig.add_subplot(gs[0, 0])


sns.heatmap(model_eval, cmap=cmap,annot=True,fmt=".2%", linewidths=5,cbar=False,ax=ax0,annot_kws={"fontsize":12})
ax0.tick_params(axis=u'both', which=u'both',length=0)

ax0.tick_params(axis='both', colors=sub_col)
ax0.tick_params(axis=u'both', which=u'both',length=0)
ax0.set_ylabel('') 

ax0.text(0,-0.75,'Model performance overview',fontsize=title,fontweight=b,color=col)
ax0.text(0,-0.2,'Both tree-based methods, Random Forest and\nXGBoost, performed the best. Logistic Regression\nalso performed well.',fontsize=sub,color=sub_col)

plt.show()

Random Forest is the best performing model so far.

Let's try stacking our three best performing models

In [ ]:
# Model stacking

scv=StackingCVClassifier(classifiers=[xgb,rf,lr],meta_classifier= rf,random_state=42)
scv.fit(X_train,y_train)
scv_predicted = scv.predict(X_test)
scv_conf_matrix = confusion_matrix(y_test, scv_predicted)
scv_acc_score = accuracy_score(y_test, scv_predicted)
print("confussion matrix")
print(scv_conf_matrix)
print("\n")
print("Accuracy of StackingCVClassifier:",scv_acc_score*100,'\n')
print(classification_report(y_test,scv_predicted))

# ... work in progress